In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution3D,Conv3D,Add,Concatenate, MaxPool3D,LeakyReLU,add, Convolution2D,BatchNormalization,AveragePooling3D, GlobalAveragePooling3D, ZeroPadding3D
from keras import backend as K
import cv2
import operator
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
import nibabel as nib
import pandas as pd
from scipy.ndimage import rotate
import random
import nibabel as nib
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
import nibabel as nib
from matplotlib import animation, rc
from scipy.ndimage import rotate
import glob
import os
from sklearn.model_selection import KFold, StratifiedKFold,train_test_split
from scipy.ndimage import zoom
from matplotlib.patches import PathPatch, Rectangle
from IPython.display import HTML
from tensorflow.keras import layers
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

print("Using GPU:", gpus)


In [ ]:
luna_df = pd.read_excel('/home/m-health/TBM/Lung/final_Luna16.xlsx')
column_luna = 'malignancy'
luna_series=luna_df['Series Uid'].tolist()

luna_folder = '/home/m-health/TBM/Lung/Luna_16_hu_from_crop_32'  
luna = []
for i in luna_series:
    nii_file_path = os.path.join(luna_folder, f'{i}.nii')  # Assuming NII files have the extension '.nii'
    
    if os.path.exists(nii_file_path):
        luna.append(i)
luna=np.array(luna)

print(luna.shape)

# Specify the path to the Excel file
file_path = '/home/m-health/TBM/Lung/test_initialtrain_vai_set.xlsx'
df = pd.read_excel(file_path)

z_train = df.iloc[:, 2]  

z_test_fixed = df.iloc[:, 0]  
nan_count = z_test_fixed.isna().sum()
z_test = z_test_fixed.dropna()

z_val_fixed = df.iloc[:, 1]  
nan_count_val = z_val_fixed.isna().sum()
z_val = z_val_fixed.dropna()

In [ ]:
BS= 16 # batch size
E = 200 #epoch

In [ ]:
def aug_rotate(x):

    rotated_slices = []
    for slice_idx in range(x.shape[2]):
        slice_data = x[:, :, slice_idx]  # Extract a single slice
        rotated_slice = rotate(slice_data, angle=angles[j], reshape=False, mode='nearest')
        rotated_slices.append(rotated_slice)

    # Create a new NIfTI image from the rotated slices
    rotated_img_data = np.stack(rotated_slices, axis=-1)
    # Create a new NIfTI image using the header information from the original image
    rotated_nifti = nib.Nifti1Image(rotated_img_data, img.affine)
    
    return rotated_nifti

In [ ]:
angles = [0, 45, 90, 135, 180, 225, 270, 315]



datas_test = []
labels_test = []
scans_test = []

datas_train = []
labels_train = []
scans_train = []

datas_val = []
labels_val = []
scans_val = []


for i in z_train:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        for j in range(len(angles)):
            rotated_nifti = aug_rotate(data)
            datas_train.append(rotated_nifti.get_fdata())
            labels_train.append(lab)
                    
for i in z_val:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_val.append(data)
        labels_val.append(lab)

for i in z_test:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_test.append(data)
        labels_test.append(lab)
            
X_train = np.array(datas_train)
y_train = np.array(labels_train)
X_test = np.array(datas_test)
y_test = np.array(labels_test)
X_val = np.array(datas_val)
y_val = np.array(labels_val)


X_train = np.array(datas_train)
y_train = np.array(labels_train)
X_test = np.array(datas_test)
y_test = np.array(labels_test)
X_val = np.array(datas_val)
y_val = np.array(labels_val)

print(z_train.shape, z_test.shape, z_val.shape)
print(X_train.shape, X_test.shape, X_val.shape)

# DenseNet3D

In [ ]:
from keras.layers import Input, Dense, GlobalAveragePooling3D, GlobalMaxPooling3D
from keras.models import Model
from keras import backend as K

import tensorflow as tf
from tensorflow.keras import layers, models


# -------------------------------------------------------------
# 3D Dense Layer
# -------------------------------------------------------------
def dense_layer(x, growth_rate, bottleneck=True, dropout_rate=0.0):
    """One 3D Dense Layer: BN → ReLU → Conv1x1x1 → BN → ReLU → Conv3x3x3"""
    
    if bottleneck:
        # Bottleneck 1x1x1
        x1 = layers.BatchNormalization()(x)
        x1 = layers.ReLU()(x1)
        x1 = layers.Conv3D(4 * growth_rate, kernel_size=1, use_bias=False)(x1)
    else:
        x1 = x

    # 3x3x3 conv
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.ReLU()(x1)
    x1 = layers.Conv3D(growth_rate, kernel_size=3, padding="same", use_bias=False)(x1)

    if dropout_rate:
        x1 = layers.Dropout(dropout_rate)(x1)

    # Concatenate input + output (Dense connectivity)
    return layers.Concatenate()([x, x1])


# -------------------------------------------------------------
# 3D Dense Block
# -------------------------------------------------------------
def dense_block(x, num_layers, growth_rate, bottleneck=True, dropout_rate=0.0):
    for _ in range(num_layers):
        x = dense_layer(x, growth_rate, bottleneck, dropout_rate)
    return x


# -------------------------------------------------------------
# Transition Down (Compression)
# -------------------------------------------------------------
def transition_layer(x, reduction=0.5):
    filters = int(tf.keras.backend.int_shape(x)[-1] * reduction)

    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(filters, kernel_size=1, use_bias=False)(x)
    x = layers.AveragePooling3D(pool_size=2, strides=2)(x)

    return x


# -------------------------------------------------------------
# DenseNet3D (Main constructor)
# -------------------------------------------------------------
def DenseNet3D(input_shape=(32, 32, 32, 1),
               depth=52,
               nb_dense_block=4,
               growth_rate=32,
               bottleneck=True,
               reduction=0.5,
               dropout_rate=0.0,
               include_top=True,
               pooling='avg',
               classes=1,
               activation='sigmoid'):

    """
    Builds a full 3D DenseNet model.
    depth examples: 121, 169, 201 (DenseNet variants)
    """

    # Depth configuration from original DenseNet paper
    # Formula: (depth - 4) / 6 gives layers per block (for bottleneck architecture)
    assert (depth - 4) % (nb_dense_block * 2) == 0, "Invalid depth for DenseNet"
    layers_per_block = (depth - 4) // (nb_dense_block * 2)

    inputs = layers.Input(shape=input_shape)

    # Initial convolution
    x = layers.Conv3D(64, kernel_size=7, strides=2, padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling3D(pool_size=3, strides=2, padding="same")(x)

    # Build Dense Blocks
    for block in range(nb_dense_block):
        x = dense_block(x, layers_per_block, growth_rate, bottleneck, dropout_rate)

        if block != nb_dense_block - 1:
            x = transition_layer(x, reduction)

    # Final BatchNorm
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Classification head
    if include_top:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling3D()(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling3D()(x)
        else:
            raise ValueError("pooling must be 'avg' or 'max'")

        x = layers.Dense(classes, activation=activation)(x)

    model = models.Model(inputs, x, name=f"DenseNet3D_{depth}")
    return model



In [ ]:
save_dir = "/home/m-health/TBM/Lung/"

# ------------------------------------------------------------
strategy = tf.distribute.get_strategy()

# ------------------------------------------------------------
# 3) Build & compile model
# ------------------------------------------------------------
with strategy.scope():
    model = DenseNet3D(
    input_shape=(32,32,32,1),
    depth=52,            # DenseNet-121
    classes=1,            # benign vs malignant
    pooling='avg',
    activation='sigmoid'
)
    model.compile(
    optimizer='adam',
    loss='binary_crossentropy',   # or binary_crossentropy if classes=1
    metrics=['accuracy']
)


# ------------------------------------------------------------
# 4) Callbacks
# ------------------------------------------------------------
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6
)

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(save_dir, "densenet.h5"),
    monitor='val_accuracy',
    save_best_only=True
)


# ------------------------------------------------------------
# 6) Train
# ------------------------------------------------------------

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=E,
    batch_size=BS,
    callbacks=[lr_callback, checkpoint_callback],
    verbose=1
)

# ------------------------------------------------------------
# 7) Load best model & evaluate
# ------------------------------------------------------------
best_model = tf.keras.models.load_model(os.path.join(save_dir, "densenet.h5"))
loss, accuracy = best_model.evaluate(X_test, y_test)

# ------------------------------------------------------------
# 8) Predictions
# ------------------------------------------------------------
predictions = best_model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)

class_names = ['benign', 'malignant']
print(classification_report(y_test, predicted_classes, target_names=class_names, digits=5))

auc = roc_auc_score(y_test, predictions)
print("AUC:", auc)

# ------------------------------------------------------------
# 9) Confusion Matrix
# ------------------------------------------------------------
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Loss: {loss*100:.2f}')

# ConvNeXtTiny

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# -----------------------------------------------------------
# Smallest ConvNeXt 3D block
# -----------------------------------------------------------
def convnext_block_3d_tiny(x, dim):
    shortcut = x

    # Depthwise conv
    x = layers.Conv3D(dim, kernel_size=7, padding="same", groups=dim)(x)

    x = layers.LayerNormalization(epsilon=1e-6)(x)

    # No huge expansion — ONLY 1x
    x = layers.Conv3D(dim, kernel_size=1)(x)
    x = layers.Activation("gelu")(x)

    # Residual
    return layers.Add()([shortcut, x])

# -----------------------------------------------------------
# Ultra-reduced ConvNeXt for 32³ nodules
# -----------------------------------------------------------
def ConvNeXt3D_reduced(input_shape=(32,32,32,1)):
    
    dims = [16, 32, 64, 128]   # drastically reduced
    depths = [1, 1, 2, 1]      # minimal blocks

    inputs = layers.Input(input_shape)
    x = inputs

    # Stem
    x = layers.Conv3D(dims[0], kernel_size=4, strides=4, padding="same")(x)

    # 4 stages
    for i in range(4):
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        for _ in range(depths[i]):
            x = convnext_block_3d_tiny(x, dims[i])

        if i < 3:
            x = layers.Conv3D(dims[i+1], kernel_size=2, strides=2, padding="same")(x)

    # global pooling
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    return models.Model(inputs, outputs)



In [ ]:
save_dir = "/home/m-health/TBM/Lung/"

# ------------------------------------------------------------
strategy = tf.distribute.get_strategy()

# ------------------------------------------------------------
# 3) Build & compile model

# Example usage with TPU/strategy
strategy = tf.distribute.get_strategy()
with strategy.scope():
    model = ConvNeXt3D_reduced()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ------------------------------------------------------------
# 4) Callbacks
# ------------------------------------------------------------
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6
)

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(save_dir, "ConvNeXt.h5"),
    monitor='val_accuracy',
    save_best_only=True
)


# ------------------------------------------------------------
# 6) Train
# ------------------------------------------------------------

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=E,
    batch_size=BS,
    callbacks=[lr_callback, checkpoint_callback],
    verbose=1
)

# ------------------------------------------------------------
# 7) Load best model & evaluate
# ------------------------------------------------------------
best_model = tf.keras.models.load_model(os.path.join(save_dir, "ConvNeXt.h5"))
loss, accuracy = best_model.evaluate(X_test, y_test)

# ------------------------------------------------------------
# 8) Predictions
# ------------------------------------------------------------
predictions = best_model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)

class_names = ['benign', 'malignant']
print(classification_report(y_test, predicted_classes, target_names=class_names, digits=5))

auc = roc_auc_score(y_test, predictions)
print("AUC:", auc)

# ------------------------------------------------------------
# 9) Confusion Matrix
# ------------------------------------------------------------
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Loss: {loss*100:.2f}')

# EfficientNet

In [ ]:
# EfficientNetV2B0 3D Keras Implementation
from keras import layers, models
import math

import tensorflow as tf
from tensorflow.keras import layers, Model

# ---------------------
# MBConv (EfficientNet block)
# ---------------------
def MBConv3D(inputs, expand_ratio, filters, stride):
    in_channels = inputs.shape[-1]
    expanded_channels = in_channels * expand_ratio

    # 1. Expansion
    if expand_ratio != 1:
        x = layers.Conv3D(expanded_channels, 1, padding="same")(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("swish")(x)
    else:
        x = inputs

    # 2. Depthwise Conv
    x = layers.Conv3D(expanded_channels, 3, strides=stride, padding="same", groups=expanded_channels)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)

    # 3. Squeeze & Excitation
    se = layers.GlobalAveragePooling3D()(x)
    se = layers.Dense(expanded_channels // 4, activation="swish")(se)
    se = layers.Dense(expanded_channels, activation="sigmoid")(se)
    se = layers.Reshape((1, 1, 1, expanded_channels))(se)
    x = layers.Multiply()([x, se])

    # 4. Projection
    x = layers.Conv3D(filters, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Skip connection
    if stride == 1 and in_channels == filters:
        x = layers.Add()([inputs, x])
    return x


# ---------------------
# EfficientNet-B0 (3D) – Simplified for small volume (32³)
# ---------------------
def EfficientNet3D_B0(input_shape=(32,32,32,1), num_classes=1):
    inputs = layers.Input(shape=input_shape)

    # Stem
    x = layers.Conv3D(16, 3, strides=1, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("swish")(x)

    # MBConv blocks (extremely reduced for 32³)
    x = MBConv3D(x, expand_ratio=1, filters=16, stride=1)
    x = MBConv3D(x, expand_ratio=6, filters=24, stride=2)
    x = MBConv3D(x, expand_ratio=6, filters=40, stride=2)
    x = MBConv3D(x, expand_ratio=6, filters=80, stride=1)

    # Head
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dropout(0.2)(x)

    # Binary classification
    if num_classes == 1:
        outputs = layers.Dense(1, activation="sigmoid")(x)
    else:
        outputs = layers.Dense(2, activation="softmax")(x)

    return Model(inputs, outputs)






In [ ]:
save_dir = "/home/m-health/TBM/Lung/"

# ------------------------------------------------------------
strategy = tf.distribute.get_strategy()

# ------------------------------------------------------------
# 3) Build & compile model

# Example usage with TPU/strategy
strategy = tf.distribute.get_strategy()
with strategy.scope():
    model = EfficientNet3D_B0(input_shape= (32,32,32,1))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ------------------------------------------------------------
# 4) Callbacks
# ------------------------------------------------------------
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6
)

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(save_dir, "EfficientNet.h5"),
    monitor='val_accuracy',
    save_best_only=True
)


# ------------------------------------------------------------
# 6) Train
# ------------------------------------------------------------

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=E,
    batch_size=BS,
    callbacks=[lr_callback, checkpoint_callback],
    verbose=1
)

# ------------------------------------------------------------
# 7) Load best model & evaluate
# ------------------------------------------------------------
best_model = tf.keras.models.load_model(os.path.join(save_dir, "EfficientNet.h5"))
loss, accuracy = best_model.evaluate(X_test, y_test)

# ------------------------------------------------------------
# 8) Predictions
# ------------------------------------------------------------
predictions = best_model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)

class_names = ['benign', 'malignant']
print(classification_report(y_test, predicted_classes, target_names=class_names, digits=5))

auc = roc_auc_score(y_test, predictions)
print("AUC:", auc)

# ------------------------------------------------------------
# 9) Confusion Matrix
# ------------------------------------------------------------
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Loss: {loss*100:.2f}')

# MobileNet V2

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

import tensorflow as tf
from tensorflow.keras import layers, models

class DepthwiseConv3D(layers.Layer):
    def __init__(self, kernel_size, strides=1, padding='same', depth_multiplier=1, **kwargs):
        super().__init__(**kwargs)
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        self.depth_multiplier = depth_multiplier

    def build(self, input_shape):
        self.depthwise_conv = layers.Conv3D(
            filters=int(input_shape[-1] * self.depth_multiplier),
            kernel_size=self.kernel_size,
            strides=self.strides,
            padding=self.padding,
            groups=input_shape[-1],
            use_bias=False
        )

    def call(self, inputs):
        return self.depthwise_conv(inputs)

def inverted_res_block(x, expansion, stride, alpha, filters, block_id):
    in_channels = x.shape[-1]
    pointwise_filters = int(filters * alpha)

    # Expansion
    if expansion != 1:
        x_exp = layers.Conv3D(expansion * in_channels, 1, padding='same', use_bias=False)(x)
        x_exp = layers.BatchNormalization()(x_exp)
        x_exp = layers.ReLU(6.)(x_exp)
    else:
        x_exp = x

    # Depthwise
    x_exp = DepthwiseConv3D(kernel_size=3, strides=stride, padding='same')(x_exp)
    x_exp = layers.BatchNormalization()(x_exp)
    x_exp = layers.ReLU(6.)(x_exp)

    # Projection
    x_proj = layers.Conv3D(pointwise_filters, 1, padding='same', use_bias=False)(x_exp)
    x_proj = layers.BatchNormalization()(x_proj)

    # Skip connection
    if stride == 1 and in_channels == pointwise_filters:
        x_proj = layers.Add()([x, x_proj])

    return x_proj

def MobileNetV2_3D(input_shape=(32,32,32,1), alpha=1.0, classes=1):
    inputs = layers.Input(shape=input_shape)
    first_filters = int(32 * alpha)

    # Stem
    x = layers.Conv3D(first_filters, 3, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU(6.)(x)   # 32 → 16

    # Blocks
    x = inverted_res_block(x, expansion=1, stride=1, alpha=alpha, filters=16, block_id=0)
    x = inverted_res_block(x, expansion=6, stride=2, alpha=alpha, filters=24, block_id=1) # 16 → 8
    x = inverted_res_block(x, expansion=6, stride=1, alpha=alpha, filters=24, block_id=2)
    x = inverted_res_block(x, expansion=6, stride=2, alpha=alpha, filters=32, block_id=3) # 8 → 4
    x = inverted_res_block(x, expansion=6, stride=1, alpha=alpha, filters=32, block_id=4)
    x = inverted_res_block(x, expansion=6, stride=1, alpha=alpha, filters=32, block_id=5)

    # Final Conv
    last_filters = 1280 if alpha <= 1 else int(1280 * alpha)
    x = layers.Conv3D(last_filters, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU(6.)(x)

    # Classifier
    x = layers.GlobalAveragePooling3D()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)  # binary

    return models.Model(inputs, outputs)




In [ ]:
save_dir = "/home/m-health/TBM/Lung/"

# ------------------------------------------------------------
strategy = tf.distribute.get_strategy()

# ------------------------------------------------------------
# 3) Build & compile model

with strategy.scope():
    model = MobileNetV2_3D(input_shape=(32,32,32,1), alpha=1.0, classes=1)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ------------------------------------------------------------
# 4) Callbacks
# ------------------------------------------------------------
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6
)

checkpoint_callback = ModelCheckpoint(
    # filepath=os.path.join(save_dir, "MobileNetV2.h5"),
    filepath=os.path.join(save_dir, "MobileNetV2"),
    save_format="tf",
    monitor='val_accuracy',
    save_best_only=True
)


# ------------------------------------------------------------
# 6) Train
# ------------------------------------------------------------

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=E,
    batch_size=BS,
    callbacks=[lr_callback, checkpoint_callback],
    verbose=1
)

# ------------------------------------------------------------
# 7) Load best model & evaluate
# ------------------------------------------------------------
# best_model = tf.keras.models.load_model(os.path.join(save_dir, "MobileNetV2.h5"))
best_model = tf.keras.models.load_model(os.path.join(save_dir, "MobileNetV2"))

loss, accuracy = best_model.evaluate(X_test, y_test)

# ------------------------------------------------------------
# 8) Predictions
# ------------------------------------------------------------
predictions = best_model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)

class_names = ['benign', 'malignant']
print(classification_report(y_test, predicted_classes, target_names=class_names, digits=5))

auc = roc_auc_score(y_test, predictions)
print("AUC:", auc)

# ------------------------------------------------------------
# 9) Confusion Matrix
# ------------------------------------------------------------
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Loss: {loss*100:.2f}')

# VGG-19

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def VGG19_3D_reduced(input_shape=(32,32,32,1), num_classes=1):
    inputs = layers.Input(shape=input_shape)

    # -------------------------
    # Block 1
    # -------------------------
    x = layers.Conv3D(32, 3, padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(32, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool3D(pool_size=2)(x)  # 32 -> 16

    # -------------------------
    # Block 2
    # -------------------------
    x = layers.Conv3D(64, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(64, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool3D(pool_size=2)(x)  # 16 -> 8

    # -------------------------
    # Block 3
    # -------------------------
    x = layers.Conv3D(128, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(128, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(128, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(128, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool3D(pool_size=2)(x)  # 8 -> 4

    # -------------------------
    # Block 4
    # -------------------------
    x = layers.Conv3D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool3D(pool_size=2)(x)  # 4 -> 2

    # -------------------------
    # Block 5
    # -------------------------
    x = layers.Conv3D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv3D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    # No pooling here (too small after 5th block)

    # -------------------------
    # Classifier
    # -------------------------
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation="sigmoid")(x)  # binary

    model = models.Model(inputs, outputs)
    return model
    


In [ ]:
save_dir = "/home/m-health/TBM/Lung/"

# ------------------------------------------------------------
strategy = tf.distribute.get_strategy()

# ------------------------------------------------------------
# 3) Build & compile model

# Example usage with TPU/strategy
strategy = tf.distribute.get_strategy()
with strategy.scope():
    model = VGG19_3D_reduced()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ------------------------------------------------------------
# 4) Callbacks
# ------------------------------------------------------------
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6
)

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(save_dir, "VGG19.h5"),
    monitor='val_accuracy',
    save_best_only=True
)


# ------------------------------------------------------------
# 6) Train
# ------------------------------------------------------------

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=E,
    batch_size=BS,
    callbacks=[lr_callback, checkpoint_callback],
    verbose=1
)

# ------------------------------------------------------------
# 7) Load best model & evaluate
# ------------------------------------------------------------
best_model = tf.keras.models.load_model(os.path.join(save_dir, "VGG19.h5"))
loss, accuracy = best_model.evaluate(X_test, y_test)

# ------------------------------------------------------------
# 8) Predictions
# ------------------------------------------------------------
predictions = best_model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)

class_names = ['benign', 'malignant']
print(classification_report(y_test, predicted_classes, target_names=class_names, digits=5))

auc = roc_auc_score(y_test, predictions)
print("AUC:", auc)

# ------------------------------------------------------------
# 9) Confusion Matrix
# ------------------------------------------------------------
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Loss: {loss*100:.2f}')